### 유니티와 핸드트래킹 연동
![img](https://i.ibb.co/hcvnmN7/handtracking.png)
- cvzone
  - openCV와 mediapipe를 이용한 컴퓨터 비전 모듈
- 과정
  - cvzone을 이용하여 핸드트래킹하기
  - 핸드트래킹한 랜드마크 데이터를 UDP로 유니티에게 보내기<br/><br/>
- [참고한 유튜브 - Murtaza's Workshop](https://www.youtube.com/watch?v=RQ-2JWzNc6k)

In [1]:
%pip install mediapipe opencv-python --user
%pip install cvzone --user

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import socket # UDP 프로토콜을 이용해서 유니티로 데이터를 보낸다


# 파라미터
width, height = 1280,720

# 웹캠 세팅
cap = cv2.VideoCapture(cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# 핸드 디텍터
detetctor = HandDetector(maxHands=1, detectionCon=0.8) # 핸드일 확률이 0.8 이상일 때 디텍팅

# UDP 커뮤니케이션
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # TCP는 socket.SOCK_STREAM
serverAddresssPort = ('127.0.0.1', 5052) # (로컬호스트, 포트) 사용되지 않는 포트를 이용하자!

while cap.isOpened():
    # 웹캠에서 프레임 읽어오기
    success, img = cap.read() 
    
    # 디텍터로 핸드 찾기
    hands, img = detetctor.findHands(img) 
    
    # 핸드가 찾아지면
    if hands: 
        # 랜드마크 리스트
        data = []
        
        # 첫번째 핸드만 추적 
        hand = hands[0] 
        
        # 랜드마크 리스트를 만들어서 유니티에 보낼 예정, 랜드마크 값 - (x,y,z)*21개
        lmList = hand['lmList']  
        # print(lmList) # [[123,123,123],[123,123,123],...]
        
        # x,y,z를 감싼 대괄호를 지우고 리스트 하나로 만들어서 유니티로 보내기
        for lm in lmList:
            # 파이썬과 유니티는 y값을 반대로 측정하기 때문에 스크린 최대높이에서 현재 y값 빼주기
            lmData = [-lm[0],height-lm[1],lm[2]]
            data.extend(lmData) 
        # print(data) # [301, 232, 0, 339, ...]
        
         # 스트링으로 인코딩해서 소켓의 주소와 포트로 데이터 보내기 
        sock.sendto(str.encode(str(data)), serverAddresssPort)
    
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Done")
        break

cap.release()
cv2.destroyAllWindows()

Done


### 유니티 UDP스크립트
- 포트번호만 바꿔서 사용하면 된다
1. 아래 코드 복사해서 스크립트 파일 만들기
2. 유니티에서 Manager라는 empty object 생성 후 스크립트 드래그해서 넣어주기
3. 파이썬과 유니티 동시 실행
4. 유니티 Inspector에서 Data를 통해 보내진 리스트 확인가능 
```c#
using UnityEngine;
using System;
using System.Text;
using System.Net;
using System.Net.Sockets;
using System.Threading;

public class UDPReceive : MonoBehaviour
{

    Thread receiveThread;
    UdpClient client; 
    public int port = 5052;
    public bool startRecieving = true;
    public bool printToConsole = false;
    public string data;

    // 스크립트가 시행되고 제일 처음 시행되는 함수
    public void Start()
    {
        // 쓰레드 생성
        receiveThread = new Thread(
            new ThreadStart(ReceiveData)); 
        // 백그라운드에서 실행
        receiveThread.IsBackground = true;
        receiveThread.Start();
    }

    // UdpClient 인스턴스 생성 함수
    private void ReceiveData()
    {

        // 특정 포트번호로 UdpClinet 생성
        client = new UdpClient(port);
        while (startRecieving)
        {

            try
            {
                // 어떤 ip이든 수신
                IPEndPoint anyIP = new IPEndPoint(IPAddress.Any, 0);
                // 바이트어레이형태로 데이터 수신
                byte[] dataByte = client.Receive(ref anyIP); // 수신하는 데이터의 타입
                // UTF8인코딩
                data = Encoding.UTF8.GetString(dataByte);
                if (printToConsole) { print(data); }
            }
            catch (Exception err)
            {
                print(err.ToString());
            }
        }
    }

}


```


### HandTracking.cs
- 랜드마크 스크립트
```c#
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class HandTracking : MonoBehaviour
{
    // UDP 리시버
    public UDPReceive udpReceive;
    // 포인트 오브젝트 담는 리스트
    public GameObject[] handPoints; 

    void Start()
    {
        
    }

    void Update()
    {
        // UDP에서 받아온 data
        string data = udpReceive.data; 

        // 데이터가 인지되면
        if ( data.Length != 0 ) 
        {
            // data의 양쪽 대괄호 지우기
            data = data.Trim(new char[]{'[',']'});
            //print(data);

            // 쉼표를 기준으로 데이터 하나씩 가져오기
            string[] points = data.Split(',');
            print(points[0]);

            // 스트링 데이터를 플로트로 만들기
            for(int i = 0; i < 21; i++)
            {
                // x,y,z값 가져와서 플로트화하고 스케일링
                // 캠화면과 유니티 화면이 1:1로 매칭되지 않기 때문에 알맞은 수치를 찾아야 함
                float x = float.Parse(points[i*3])/80-8; 
                float y = float.Parse(points[i*3+1])/80-5; 
                float z = float.Parse(points[i*3+2])/80; 

                // 포인트를 랜드마크 위치에 놓기
                handPoints[i].transform.localPosition = new Vector3(x,y,z);
            }

        }

    }
}


```

### LineCode.cs
- 핸드라인 스크립트
```c#
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class LineCode : MonoBehaviour
{
    // 라인컴포넌트 받는 변수
    LineRenderer lineRenderer;

    // 랜드마크 위치에 있는 포인트를 서로 이어주는 변수
    public Transform origin;
    public Transform destination;

    void Start()
    {
        // 라인 컴포넌트 인스턴스를 받아오기
        lineRenderer = GetComponent<LineRenderer>();
        // 라인 두께
        lineRenderer.startWidth = 0.1f;
        lineRenderer.endWidth = 0.1f;
    }

    void Update()
    {
        // 움직이는 랜드마크 따라 위치 업데이트하기
        lineRenderer.SetPosition(0, origin.position);
        lineRenderer.SetPosition(1, destination.position);
    }
}

```